In [ ]:
""" Quick analysi of neural data, 6/22, for quick look for K99 ,
mainly for making rasters, pulling out interesting units, etc
"""

##### Load neural and beh data (pending: cam)


In [1]:
from neuralmonkey.classes.session import Session
import matplotlib.pyplot as plt
from neuralmonkey.scripts.load_and_save_locally import load_and_preprocess_single_session

import neuralmonkey.utils.monkeylogic as mkl

In [ ]:
load_and_preprocess_single_session(220703, 1)

In [2]:
animal = "Pancho"
expt = "*"
# DATE = "220608"
DATE = "220713"
rec_session = 0 # assumes one-to-one mapping between neural and beh sessions.
# dataset_beh_expt = "charnovel3"
# dataset_beh_expt = "charneuraltrain1"
dataset_beh_expt = "priminvar"

if DATE=="220532":
    # For K99, quick analy
    # Beh (ml2)
    beh_expt_list = ["charneuraltrain1b"]
    beh_sess_list = [1]
    beh_trial_map_list = [(1,0)]
    sites_garbage = [20, 26, 36, 44, 46, 56, 67, 69, 78, 98, 100, 
                          1+256, 2+256, 10+256, 91+256, 127+256]    
    SN = Session(DATE, beh_expt_list, beh_sess_list, beh_trial_map_list, sites_garbage=sites_garbage)
else:
    # General purpose
    beh_session = rec_session+1 # 1-indexing.
    sessdict = mkl.getSessionsList(animal, datelist=[DATE])
    if False:
        # get all sessions
        beh_expt_list = [sess_expt[1] for sess_expt in sessdict[DATE]]
        beh_sess_list = [sess_expt[0] for sess_expt in sessdict[DATE]]
    else:
        # Get the single session assued to map onto this neural.
        beh_expt_list = [sess_expt[1] for sess_expt in sessdict[DATE] if sess_expt[0]==beh_session]
        assert(len(beh_expt_list))==1, "must be error, multiple sessions with same session num"
        beh_sess_list = [beh_session]

    beh_trial_map_list = [(1, 0)]
    sites_garbage = None

    print("Loading these beh expts:", beh_expt_list)
    print("Loading these beh sessions:",beh_sess_list)
    print("Loading this neural session:", rec_session)

    SN = Session(DATE, beh_expt_list, beh_sess_list, beh_trial_map_list, sites_garbage=sites_garbage,
                rec_session = rec_session)

1
Loading these beh expts: ['primsingrid2']
Loading these beh sessions: [1]
Loading this neural session: 0
Searching using this string:
/mnt/hopfield_data01/ltian/recordings/*Pancho*/*220713*/**
Found this many paths:
0


AssertionError: maybe you didn't mount server?

In [5]:
# get all trials spikes (takes a while...)
# SN.DatAll = None
# SN._MapperSiteTrial2DatAllInd = {}

SN.extract_raw_and_spikes_helper()

** Loading datall from local (previusly cached)
** Loading _MapperSiteTrial2DatAllInd from local (previusly cached)
Saving DatAll (raw and spikes) locally to:  /data3/recordings/Pancho/220608/Pancho-220608-143122/data_datall.pkl
Saving _MapperSiteTrial2DatAllInd locally to:  /data3/recordings/Pancho/220608/Pancho-220608-143122/mapper_st2dat.pkl


In [6]:
SAVEDIR = f"/data2/analyses/recordings/NOTEBOOKS/quickanaly_220603/{SN.Animal}/{SN.Date}/{SN.Paths['final_dir_name']}"
import os
print(SAVEDIR)

/data2/analyses/recordings/NOTEBOOKS/quickanaly_220603/Pancho/220608/Pancho-220608-143122


##### Extract raw data for a single trial

NOTE: if want to consider all trial, ignore raw, too large

In [ ]:
trialtdt = 5

SN.extract_raw_and_spikes([2, 3], list(range(1, 257)), trialtdt, False)
# SN.extract_raw_and_spikes([2, 3], list(range(1, 257)), trialtdt, False);

In [ ]:
SN.print_summarize_datall()

##### Plot all for this trial, including raw neural 

In [ ]:
# %matplotlib notebook
%matplotlib inline
SN.plot_specific_trial_overview(trialtdt)

##### Single trial: Plot all rasters and beh and drawing

In [ ]:
sdir = f"{SAVEDIR}/PLOTS/rasters_singletrial_allsites"
os.makedirs(sdir, exist_ok=True)

In [ ]:
trial = 625
sites = SN.sitegetter_all()
# sites = SN.sitegetter_all(["M1_m", "M1_l"])
fig, axes = SN.plot_raster_oneetrial_multsites(trial, sites, WIDTH=15, HEIGHT=14)
fig.savefig(f"{sdir}/trial_{trial}.pdf")
fig.savefig(f"{sdir}/trial_{trial}.jpg")


In [ ]:
# Given list of sites (yaxis), modify plot to indicate the brain region


##### Compute and normalize FR (diff ways) for easier comparison across units


In [ ]:
# Pick out a few trials
list_trials_ml2 = [5, 10, 20, 50, 100, 110, 316, 327, 329,  622, 624, 626, 608, 612, 674]
list_trials = [x-1 for x in list_trials_ml2]
for t in list_trials:
    SN.extract_raw_and_spikes([2,3], list(range(1,257)), t, get_raw=False)
    

In [ ]:
# Pick out range of many trials

list_trials_good = [315, 326, 328,  621, 623, 625, 607, 611, 673]
list_trials_others = [t-1 for t in list_trials_good]
list_trials_others2 = [t-2 for t in list_trials_good]
list_trials = list_trials_good + list_trials_others + list_trials_others2
print(list_trials)
for t in list_trials:
    SN.extract_raw_and_spikes([2,3], list(range(1,257)), t, get_raw=False)
    

In [ ]:
# Even larger range of trials
list_trials = range(300, 700)
print(list_trials)
for t in list_trials:
    SN.extract_raw_and_spikes([2,3], list(range(1,257)), t, get_raw=False)


## PLOTS

##### One plot per trial and brain region

In [ ]:
SAVEDIR = f"/data2/analyses/recordings/NOTEBOOKS/quickanaly_220603/{SN.Animal}/{SN.Date}/{SN.Animal}-{SN.Date}-{SN.Paths['time']}"
import os
print(SAVEDIR)

In [ ]:
sdir = f"{SAVEDIR}/PLOTS/rasters_trial_by_region_all"
os.makedirs(sdir, exist_ok=True)

In [ ]:
bregion_mapper = SN.sitegetter_brainregion("mapper", clean=True)
for trial in list_trials:
    for region, sites in bregion_mapper.items():
        try:
            # sites = SN.sitegetter_brainregion("FP_p")
            # sites = SN.sitegetter_brainregion("dlPFC_a")
            # sites = SN.sitegetter_brainregion("preSMA_a")
            fig, axes = SN.plot_raster_onetrial_multsites(trial, sites)

            fig.savefig(f"{sdir}/{region}-trial_{trial}.pdf")
        except Exception as err:
            pass
        


##### One plot per trial and hand-picked units across regions

##### One plot for a unit across trials,

In [ ]:
SORT_TRIALS_BY = "duration"

site = 177 # vlpfc
# site = 312
list_trials = [315, 326, 328,  338, 347, 362, 487, 554, 621, 623, 625, 607, 611, 673, 651, 658, 682, 692, 710]

if SORT_TRIALS_BY=="duration":
    # Sort trials by duration (go to done)
    def calc_trial_dur(trial):
        go = SN.behcode_extract_times("go", trial, shorthand=True, first_instance_only=True)
        done = SN.behcode_extract_times("done", trial, shorthand=True, first_instance_only=True)
        return done-go

    list_durs = [calc_trial_dur(trial) for trial in list_trials]

    tmp = [(t, d) for t, d in zip(list_trials, list_durs)]
    tmp = sorted(tmp, key=lambda x: x[1])
    list_trials = [x[0] for x in tmp]
elif SORT_TRIALS_BY=="chron":
    list_trials = sorted(list_trials)
else:
    assert False

# How to align them?
alignto = 'go'



In [ ]:
# make sure extracted all spikes for these trials
SN.extract_raw_and_spikes_helper(list_trials)

In [ ]:
dict_sites = {
    "dlPFC_a":[255, 243, 231],
    "dlPFC_p":[222, 215, 198],
    "FP_a":[97, 119],
    "FP_p":[77, 92, 95, 96], 
    "M1_m":[31, 29, 13],
    "M1_l":[39, 43, 45, 57, 63],
    "PMd_a":[175, 191, 177, 179, 181, 172, 187],
    "PMv_l":[66, 90, 72, 77],
    "preSMA_a":[227, 256, 246, 253, 246],
    "preSMA_p":[221, 212, 213, 193],
    "SMA_a":[173, 179, 181],
    "SMA_p":[151, 140],
    "vlPFC_a":[33, 47, 56, 38, 56, 64],
    "vlPFC_p":[5, 16, 22], 
}


# convert to actual sites
list_sites = []
for region, chans in dict_sites.items():
    for ch in chans:
        list_sites.append(SN.sitegetter_brainregion_chan(region, ch))
list_sites = sorted(list_sites)

In [ ]:
sdir = f"{SAVEDIR}/PLOTS/rasters_multtrials_by_site_handpicked_sortby_{SORT_TRIALS_BY}"
os.makedirs(sdir, exist_ok=True)

In [ ]:
list_alignto = ["on", "fixcue", "fix", "samp", "go", "done", "fb_vs", "rew",
                ("strokes", 0, "on"), ("strokes", -1, "off"),
                ("strokes", 0, "off")
               ]
# list_alignto = [("strokes", -1, "off"),
#                 ("strokes", 0, "off")
#                ]
saved_draw=False
for site in list_sites:
    for alignto in list_alignto:
        fig1, _, fig2, _ = SN.plot_raster_multrials_onesite(list_trials, site=site, alignto=alignto)
        
        prefix = SN.sitegetter_summarytext(site)
        fig1.savefig(f"{sdir}/{prefix}-alignto_{alignto}-rasters.pdf")
        
        if saved_draw==False:
            saved_draw=True
            fig2.savefig(f"{sdir}/drawings.pdf")
            
        plt.close("all")
        

In [ ]:
# Same, but ordering trials by the duiration (go to done)

list_alignto = ["on", "fixcue", "fix", "samp", "go", "done", "fb_vs", "rew",
                ("strokes", 0, "on"), ("strokes", -1, "off"),
                ("strokes", 0, "off")
               ]
# list_alignto = [("strokes", -1, "off"),
#                 ("strokes", 0, "off")
#                ]
saved_draw=False
for site in list_sites:
    for alignto in list_alignto:
        fig1, _, fig2, _ = SN.plot_raster_multrials_onesite(list_trials, site=site, alignto=alignto)
        
        prefix = SN.sitegetter_summarytext(site)
        fig1.savefig(f"{sdir}/{prefix}-alignto_{alignto}-rasters.pdf")
        
        if saved_draw==False:
            saved_draw=True
            fig2.savefig(f"{sdir}/drawings.pdf")
            
        plt.close("all")
        

### SMOOTHED FR

In [ ]:
# Gaussian kernel smoothing

In [ ]:
SN.DatAll[0]

In [ ]:
# 1) bin spikes


# 2) gaussian kernel


#### Using Elephant

In [ ]:

for d in SN.DatAll:
#         spike_times = datspikes_slice_single(d["rs"], d["chan"], d["time_range"])
    print(d["trial0"])
    spiketimes, time_dur, time_on, time_off = SN.datspikes_slice_single(d["rs"], d["chan"], trial0=d["trial0"])
    d["spike_times"] = spiketimes
    d["time_dur"] = time_dur
    d["time_on"] = time_on
    d["time_off"] = time_off


In [ ]:
SN.sitegetter_all(clean=False)

In [ ]:
# Get all for a single trial
trial = 600
fig, ax = plt.subplots(1,1)
SN.plot_trial_timecourse_summary(ax, trial)

In [ ]:
# Focus on one trial for now, get all sites.

trial = 600
for site in SN.sitegetter_all(clean=False):
    print(site)
    SN.spiketrain_as_elephant(site, trial)

In [ ]:
SN.spiketrain_as_elephant_batch()

In [ ]:
# Get all spike trains for a trial

list_sites = SN.sitegetter_all(clean=False)
list_spiketrain = []
for site in list_sites:
    SN.spiketrain_as_elephant(site, trial)
    dat = SN.datall_slice_single_bysite(site,trial)
    list_spiketrain.append(dat["spiketrain"])


In [ ]:
from elephant.kernels import GaussianKernel
frate = instantaneous_rate(list_spiketrain, sampling_period=0.01*s, kernel=GaussianKernel(0.1*s))

In [ ]:
frate.shape

In [ ]:
frate.T.shape

##### Scratch (for smoothing spikes)

In [ ]:
plt.figure(dpi=150)

# # plotting the original spiketrain
# plt.plot(st_train, [0]*len(st_train), 'r', marker=2, ms=25, markeredgewidth=2, lw=0, label='poisson spike times')

# # mean firing rate
# plt.hlines(mean_firing_rate(st_train), xmin=st_train.t_start, xmax=st_train.t_stop, linestyle='--', label='mean firing rate')

# # time histogram
# plt.bar(histthis.times, histthis.magnitude.flatten(), width=histthis.sampling_period, align='edge', alpha=0.3, label='time histogram (rate)')

# instantaneous rate
# plt.plot(frate.times, frate.rescale(histthis.dimensionality).magnitude.flatten(), label='instantaneous rate')
plt.plot(frate.times, frate.magnitude[:,site], label='instantaneous rate')

# axis labels and legend
plt.xlim(frate.t_start, frate.t_stop)
plt.legend()
plt.show()

In [ ]:
# Compute histogram

from elephant.statistics import time_histogram, instantaneous_rate,mean_firing_rate
from quantities import s

bindur = 0.1 * s
histthis = time_histogram(list_spiketrain, bindur, output="rate")

print(type(histthis), f"of shape {histthis.shape}: {histthis.shape[0]} samples, {histthis.shape[1]} channel")
print('sampling rate:', histthis.sampling_rate)
print('times:', histthis.times)
print('counts:', histthis.T[0])


In [ ]:
from elephant.kernels import GaussianKernel
frate = instantaneous_rate(list_spiketrain, sampling_period=0.02*s, kernel=GaussianKernel(0.1*s))

In [ ]:
frate

In [ ]:
frate.rescale(frate.dimensionality).magnitude

In [ ]:
site = 100
frate.magnitude[:,site]

# CLEANUP

##### Check spike waveforms

In [ ]:
sdir = f"{SAVEDIR}/PLOTS/spike_waveforms"
os.makedirs(sdir, exist_ok=True)


In [ ]:
SN.SitesAll

In [ ]:
waveforms.shape

In [ ]:
fig, ax = plt.subplots(1,1)
SN.plot_spike_waveform(ax, waveforms)


In [ ]:
waveforms = SN.load_spike_waveforms_(rs=2, chan=20)

In [ ]:
import numpy as np
# Plot waveforms for all channels in a grid
RS = 2
CHAN = 218
YLIM = [-250, 100]

fig, axes = plt.subplots(1,2, figsize=(12,4))

# 1) Waveforms
ax = axes.flatten()[0]
waveforms = SN.load_spike_waveforms_(rs=RS, chan=CHAN)
SN.plot_spike_waveform(ax, waveforms)
ax.set_ylim(YLIM)

# 2) min and max voltages
outdict = SN.spikewave_compute_stats(waveforms)
ax = axes.flatten()[1]
ax.hist(outdict["volt_max"], bins=100);
ax.hist(outdict["volt_min"], bins=100);



In [ ]:
SN.Paths

In [ ]:
SN.plot_spike_waveform_stats_multchans(XLIM=[-400, 200])
SN.plot_spike_waveform_multchans(LIST_YLIM=[YLIM])

In [ ]:
SN.DatSpikeWaveforms = {}

In [ ]:
def s

In [ ]:
## Collect stats about each site, to autoamtically remove bad sites

for site in SN.sitegetter_all(clean=False):
    rs, chan = SN.convert_site_to_rschan(site)
    wf = SN.load_spike_waveforms_(rs, chan)

In [ ]:
## NOTE DOWN sites with weak/no activity

In [ ]:
sites_weak = [1,2,3,4,6, 8, 18, 19, 21, 24, 25, 26,
              33, 35, 36, 38, 44, 46, 48, 50, 64, 
             65, 67, 68, 69, 71, 72, 75, 76, 77, 81, 85, 86, 87, 
              89, 92, 93, 95
              100, 104, 105, 110, 111, 115, 120, 122, 127, 128,
              129, 131, 133, 134, 137, 139, 147, 149, 152, 155, 
              156, 157, 159, 163, 166, 168, 174, 182, 185, 190, 191,
              196, 199, 201, 202, 204, 206, 208, 212, 214, 222,
              224, 230, 234, 238, 242, 251, 
              258, 266, 268, 284, 286, 289, 291, 292, 293, 294, 295, 
              297, 298, 305, 306, 307, 308, 311, 313, 315, 317, 318, 319, 
              322, 323, 324, 325, 326, 328, 329, 330, 336, 
              338, 341, 342, 343, 345, 350, 355, 357, 359, 361, 362,363, 
              364, 365,  355, 357, 359, 361, 362, 363, 364,366,
              370, 372. 375, 376, 377, 378, 379, 381, 382, 383, 384,
              391, 418, 424, 426, 427, 432, 440, 444, 446, 448,
              476, 482, 507]



In [18]:
# 220608
# sites_garbage = [(2,12), (2,21), (2,34), (2,36), (2,50), (2,67), (2,68), (2,69), 
sites_garbage = [12, 21, 26, 34, 36, 46, 48, 50, 60, 65, 67, 68, 69, 70, 73, 77, 78, 82, 83, 84, 85, 87, 88, 89, 
                 91, 92, 93, 95, 96, 97, 100, 101, 109, 115, 122, 127, 129, 131, 133,  137, 139, 142, 143, 
                 145, 147, 149, 151, 152, 155, 157, 160, 168, 182, 183, 184, 190, 192, 196, 204, 206, 210, 
                 226, 234, 250, 262, 264, 266, 268, 287, 289, 290, 291, 292, 295, 298, 304, 
                 306, 307, 308, 309, 311, 313, 315, 318, 319, 321, 322, 323, 324, 325, 326, 330, 335, 
                 337, 338, 340, 341, 342, 348, 350, 355, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366,  
                 372, 376, 378, 379, 380, 382, 383, 384, 
                 411, 413, 419, 426, 428, 432, 440, 444, 445, 446, 448,
                 462, 478, 482, 487, 507]
SN.SitesGarbage = sites_garbage

In [19]:
assert np.all(np.diff(sites_garbage)>0.), "you made mistake entering chanels (assuming going in order)"

In [ ]:
# Plot waveforms for garbage and good.

# Good:
SN.plot_spike_waveform_multchans(clean=True, LIST_YLIM=[YLIM])
SN._plot_spike_waveform_multchans(SN.SitesGarbage, YLIM, True, "garbage")

# SCRATCH

##### Integrate with beh (monkeylogic)

In [ ]:
%cd ..

from tools.utils import * 
from tools.plots import *
# from tools.analy import *
# from tools.calc import *
# from tools.analyplot import *
from tools.preprocess import *
# from tools.dayanalysis import *
# from analysis.line2 import *
# from analysis.probedatTaskmodel import *
# from pythonlib.drawmodel.analysis import *
# from pythonlib.tools.stroketools import *

In [ ]:
# Load filedata
# Load filedata, quick
a = "Pancho"
d = 220531
e = "charneuraltrain1b"
s = 1

fd = loadSingleDataQuick(a, d, e, s)

In [ ]:
data_tank = tdt.read_block(PATH_TANK, evtype = ["epocs", "streams"])

In [ ]:
if False:
    # tdt.epoc_filter(data, 'SMa1', values=[9., 18.])
    tdt.epoc_filter(data_tank, 'SMa1', t = [-0.1, 0.5])

In [ ]:
# RS4 starts at a later sample, so account for that
RS4_SAMPLE_ADD = 1 # 1 means RS4 started 1 sample later.
# TODO: add this. 

In [ ]:
extract_behcode_times_sequence([9, 11, 16, 91, 92, 62, 73, 50], trial0+1)


## PLOTS

In [ ]:
# Plot subset of chans


In [ ]:
# Label by brain region.

In [ ]:
# Remove bad channels

In [ ]:
# Plot normalized FR?

##### For each unit, plot across N (hand picked trials) and save



In [ ]:
# list chans (chans to overlay)


# ===  5/31
# for plotting exmaples, 
list_good_sites = [
    6, 7, 10, 12, 13]

# Sites that should not even inlcude in raster because of noise
# not because of low FR.
list_sites_garbage = [20, 26, 36, 44, 46, 56, 67, 69, 78, 98, 100, 
                      1+256, 2+256, 10+256, 91+256, 127+256]    



    
    
    
    
    


In [ ]:

SN.sitegetter_brainregion("M1_m")

In [ ]:
list_sites_all = range(1, 512)
list_sites_all = [s for s in list_sites_all if s not in list_sites_garbage]

print(len(list_sites_all))


In [ ]:
sdir = f"{SAVEDIR}/rasters_region_by_trials"
os.makedirs(sdir, exist_ok=True)

In [ ]:
# go thru

In [ ]:
# small set of trials, one plot for each site

plot_trial_timecourse_summary(

In [ ]:
# 1) 1 plot for each area